In [91]:
import pandas as pd
import requests
import re
from bs4 import BeautifulSoup

In [92]:
url = "http://www.espn.com/mlb/freeagents/_/year/2017/type/dollars"
response = requests.get(url)
page = response.text
# print(page)

In [93]:
soup = BeautifulSoup(page,"html5lib")

In [94]:
# print(soup.prettify())

In [95]:
rowplayer_regex = re.compile('.*row\splayer.*')
# soup.find_all(class_='oddrow player-10-30993') #example player
#Eric Hosmer - 1B signed by the San Diego Padres

In [96]:
#Players are in the table as evenrow/oddrow
#presumably for the small change in shade for formatting

In [97]:
# for link in soup.find_all(class_=rowplayer_regex): 
#     print(link)

In [98]:
#Can I grab column names?

In [99]:
col_name_all = soup.find_all(class_="colhead")[1]
col_names_list=[]
for name in col_name_all.find_all("td"):
    col_names_list.append(name.text)

In [100]:
print(col_names_list) #woohoo!

['PLAYER', 'POS', 'AGE', 'STATUS', '2017 TEAM', 'NEW TEAM', 'YRS', 'RK', 'DOLLARS']


In [101]:
#Now Lets Build a List of 2017 MLB Free Agents

In [102]:
# free_agents_list = []
# for link in soup.find_all(class_=rowplayer_regex): #just adjust to find_all 
#     free_agents_list.append(link)
    
free_agents_list = soup.find_all(class_=rowplayer_regex)

In [103]:
# print(free_agents_list[0])

In [104]:
free_agents_17 = {}

In [105]:
for row in free_agents_list:
    items = row.find_all('td')
    player = items[0].find('a').text
#     print(player)
    free_agents_17[player] = [i.text for i in items[1:]]

In [106]:
# test_agent = free_agents_list[0]
# test_tds = test_agent('td')
# test_tds

In [107]:
# free_agents_17

In [108]:
free_agents_17df = pd.DataFrame.from_dict(data=free_agents_17,orient='index')

In [109]:
free_agents_17df.reset_index(inplace=True)
# free_agents_17df.rename(columns={"index":"PLAYER","0":"POS","1"})


In [110]:
free_agents_17df.head()

,index,0,1,2,3,4,5,6,7
0,Eric Hosmer,1B,28,Signed,Royals,Padres,8,7,"$144,000,000"
1,Yu Darvish,SP,31,Signed,Dodgers,Cubs,6,1,"$126,000,000"
2,J.D. Martinez,RF,30,Signed,Diamondbacks,Red Sox,5,6,"$110,000,000"
3,Lorenzo Cain,CF,32,Signed,Royals,Brewers,5,2,"$80,000,000"
4,Jake Arrieta,SP,32,Signed,Cubs,Phillies,3,4,"$75,000,000"


In [111]:
col_names_list    

['PLAYER',
 'POS',
 'AGE',
 'STATUS',
 '2017 TEAM',
 'NEW TEAM',
 'YRS',
 'RK',
 'DOLLARS']

In [112]:
col_names_dict = {"index":"PLAYER",0:"POS",1:"AGE",2:"STATUS",\
                 3:"OLD_TEAM",4:"NEW_TEAM",5:"YRS",6:"RK"\
                  ,7:"DOLLARS"}

In [113]:
free_agents_17df.rename(columns=col_names_dict,inplace=True)

In [114]:
#Last Step - Add Free Agent Year as Column
free_agents_17df["FA_YEAR"]="2017"

In [115]:
free_agents_17df.head()

,PLAYER,POS,AGE,STATUS,OLD_TEAM,NEW_TEAM,YRS,RK,DOLLARS,FA_YEAR
0,Eric Hosmer,1B,28,Signed,Royals,Padres,8,7,"$144,000,000",2017
1,Yu Darvish,SP,31,Signed,Dodgers,Cubs,6,1,"$126,000,000",2017
2,J.D. Martinez,RF,30,Signed,Diamondbacks,Red Sox,5,6,"$110,000,000",2017
3,Lorenzo Cain,CF,32,Signed,Royals,Brewers,5,2,"$80,000,000",2017
4,Jake Arrieta,SP,32,Signed,Cubs,Phillies,3,4,"$75,000,000",2017


In [117]:
free_agents_17df["DOLLARS"] = free_agents_17df["DOLLARS"].str.replace(r'\D','0')

In [118]:
free_agents_17df.head()

,PLAYER,POS,AGE,STATUS,OLD_TEAM,NEW_TEAM,YRS,RK,DOLLARS,FA_YEAR
0,Eric Hosmer,1B,28,Signed,Royals,Padres,8,7,014400000000,2017
1,Yu Darvish,SP,31,Signed,Dodgers,Cubs,6,1,012600000000,2017
2,J.D. Martinez,RF,30,Signed,Diamondbacks,Red Sox,5,6,011000000000,2017
3,Lorenzo Cain,CF,32,Signed,Royals,Brewers,5,2,08000000000,2017
4,Jake Arrieta,SP,32,Signed,Cubs,Phillies,3,4,07500000000,2017


In [70]:
free_agents_17df["DOLLARS"]=free_agents_17df["DOLLARS"].str.replace("\$|,",'0')
free_agents_17df["DOLLARS"]=free_agents_17df["DOLLARS"].str.replace("Minor Lg",'0')
free_agents_17df["DOLLARS"]=free_agents_17df["DOLLARS"].str.replace("--",'0')
free_agents_17df.head()

,PLAYER,POS,AGE,STATUS,OLD_TEAM,NEW_TEAM,YRS,RK,DOLLARS,FA_YEAR
0,Eric Hosmer,1B,28,Signed,Royals,Padres,8,7,014400000000,2017
1,Yu Darvish,SP,31,Signed,Dodgers,Cubs,6,1,012600000000,2017
2,J.D. Martinez,RF,30,Signed,Diamondbacks,Red Sox,5,6,011000000000,2017
3,Lorenzo Cain,CF,32,Signed,Royals,Brewers,5,2,08000000000,2017
4,Jake Arrieta,SP,32,Signed,Cubs,Phillies,3,4,07500000000,2017


In [89]:
num_cols = ["DOLLARS","AGE","YRS","FA_YEAR"]
for col in num_cols:
    free_agents_17df[col]=\
    pd.to_numeric(free_agents_17df[col])

# string_cols = ["PLAYER","POS","OLD_TEAM","NEW_TEAM"]
# for col in string_cols:
#     free_agents_17df[col]=\
#     free_agents_17df[col].astype('str')

In [90]:
free_agents_17df.dtypes

PLAYER       object
POS          object
AGE           int64
STATUS       object
OLD_TEAM     object
NEW_TEAM     object
YRS         float64
RK           object
DOLLARS       int64
FA_YEAR       int64
dtype: object

## ESPN FREE AGENT DATA HAS BEEN GRABBED + CLEANED
next step with this is to build script to grab multiple years of this data
